In [21]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
appName("Sneha Spark Session").\
config("spark.shuffle.useOldFetchProtocol", 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o46.sessionState

In [2]:
spark

In [3]:
order_schema = 'order_id long, order_date date, customer_id long, order_status string'

In [4]:
order_df = spark.read.\
format("csv").\
schema(order_schema).\
load("/public/trendytech/retail_db/ordersnew")

In [5]:
order_df.show()

+--------+----------+-----------+------------+
|order_id|order_date|customer_id|order_status|
+--------+----------+-----------+------------+
|    2480|2013-08-07|       3807|    COMPLETE|
|   30479|2014-01-30|       9265|    COMPLETE|
|    2481|2013-08-07|       2476|    COMPLETE|
|   30481|2014-01-30|       9240|    COMPLETE|
|    2483|2013-08-07|      10453|    COMPLETE|
|   30484|2014-01-30|       2876|    COMPLETE|
|    2484|2013-08-07|       9256|    COMPLETE|
|   30485|2014-01-30|       1069|    COMPLETE|
|    2488|2013-08-07|       1255|    COMPLETE|
|   30486|2014-01-30|       1151|    COMPLETE|
|    2491|2013-08-07|        247|    COMPLETE|
|   30487|2014-01-30|       6772|    COMPLETE|
|    2495|2013-08-07|       9011|    COMPLETE|
|   30489|2014-01-30|       5717|    COMPLETE|
|    2498|2013-08-07|       1966|    COMPLETE|
|   30490|2014-01-30|      12189|    COMPLETE|
|    2511|2013-08-07|       8544|    COMPLETE|
|   30492|2014-01-30|       3710|    COMPLETE|
|    2515|201

In [6]:
order_df.createOrReplaceTempView("order")

In [7]:
spark.sql("select * from order").show()

+--------+----------+-----------+------------+
|order_id|order_date|customer_id|order_status|
+--------+----------+-----------+------------+
|    2480|2013-08-07|       3807|    COMPLETE|
|   30479|2014-01-30|       9265|    COMPLETE|
|    2481|2013-08-07|       2476|    COMPLETE|
|   30481|2014-01-30|       9240|    COMPLETE|
|    2483|2013-08-07|      10453|    COMPLETE|
|   30484|2014-01-30|       2876|    COMPLETE|
|    2484|2013-08-07|       9256|    COMPLETE|
|   30485|2014-01-30|       1069|    COMPLETE|
|    2488|2013-08-07|       1255|    COMPLETE|
|   30486|2014-01-30|       1151|    COMPLETE|
|    2491|2013-08-07|        247|    COMPLETE|
|   30487|2014-01-30|       6772|    COMPLETE|
|    2495|2013-08-07|       9011|    COMPLETE|
|   30489|2014-01-30|       5717|    COMPLETE|
|    2498|2013-08-07|       1966|    COMPLETE|
|   30490|2014-01-30|      12189|    COMPLETE|
|    2511|2013-08-07|       8544|    COMPLETE|
|   30492|2014-01-30|       3710|    COMPLETE|
|    2515|201

In [11]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(date_format(order_date,'MM')) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).show()

+-----------+-----------+---------+---------+
|customer_id|order_month|total_cnt|month_num|
+-----------+-----------+---------+---------+
|       1852|    January|      375|       01|
|        332|    January|     2010|       01|
|         18|    January|     4020|       01|
|       4249|    January|     2760|       01|
|        255|    January|     2010|       01|
|       1182|    January|      375|       01|
|       1606|    January|     2010|       01|
|       2787|    January|     2010|       01|
|       4318|    January|     2385|       01|
|       4618|    January|     4770|       01|
|       9905|    January|     2010|       01|
|      10148|    January|     2010|       01|
|      10209|    January|      375|       01|
|      10635|    January|     2010|       01|
|      11156|    January|      375|       01|
|      11347|    January|     2010|       01|
|       4409|    January|     2010|       01|
|       7268|    January|     2010|       01|
|       4353|    January|     2010

In [12]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(date_format(order_date,'MM')) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).write.format("noop").mode("overwrite").save()

In [13]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(int(date_format(order_date,'MM'))) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).show()

+-----------+-----------+---------+---------+
|customer_id|order_month|total_cnt|month_num|
+-----------+-----------+---------+---------+
|       4353|    January|     2010|        1|
|       6742|    January|     2010|        1|
|       5073|    January|     2385|        1|
|       1850|    January|     2010|        1|
|       5009|    January|     2010|        1|
|       7326|    January|     2010|        1|
|       3650|    January|     2010|        1|
|       2502|    January|      750|        1|
|      12424|    January|      375|        1|
|       6524|    January|     2010|        1|
|      10137|    January|      375|        1|
|        616|    January|      375|        1|
|      12164|    January|      375|        1|
|       8254|    January|      375|        1|
|       5452|    January|      375|        1|
|       3047|    January|      375|        1|
|       8698|    January|      750|        1|
|       9583|    January|      375|        1|
|       2508|    January|      375

In [16]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(int(date_format(order_date,'MM'))) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).write.format("noop").mode("overwrite").save()

In [15]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(date_format(order_date,'MM')) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).write.format("noop").mode("overwrite").save()

In [19]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(int(date_format(order_date,'MM'))) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).explain(True)

== Parsed Logical Plan ==
'Sort ['month_num ASC NULLS FIRST], true
+- 'Aggregate ['customer_id, 'order_month], ['customer_id, 'date_format('order_date, MMMM) AS order_month#325, 'count(1) AS total_cnt#326, first('int('date_format('order_date, MM)), false) AS month_num#328]
   +- 'UnresolvedRelation [order], [], false

== Analyzed Logical Plan ==
customer_id: bigint, order_month: string, total_cnt: bigint, month_num: int
Sort [month_num#328 ASC NULLS FIRST], true
+- Aggregate [customer_id#2L, date_format(cast(order_date#1 as timestamp), MMMM, Some(America/Toronto))], [customer_id#2L, date_format(cast(order_date#1 as timestamp), MMMM, Some(America/Toronto)) AS order_month#325, count(1) AS total_cnt#326L, first(cast(date_format(cast(order_date#1 as timestamp), MM, Some(America/Toronto)) as int), false) AS month_num#328]
   +- SubqueryAlias order
      +- Relation[order_id#0L,order_date#1,customer_id#2L,order_status#3] csv

== Optimized Logical Plan ==
Sort [month_num#328 ASC NULLS FIRST],

In [20]:
spark.sql("""
            select customer_id,date_format(order_date,'MMMM') as order_month,
            count(*) as total_cnt,first(date_format(order_date,'MM')) as month_num
            from order
            group by customer_id,order_month
            order by month_num      
          """).explain(True)

== Parsed Logical Plan ==
'Sort ['month_num ASC NULLS FIRST], true
+- 'Aggregate ['customer_id, 'order_month], ['customer_id, 'date_format('order_date, MMMM) AS order_month#342, 'count(1) AS total_cnt#343, first('date_format('order_date, MM), false) AS month_num#345]
   +- 'UnresolvedRelation [order], [], false

== Analyzed Logical Plan ==
customer_id: bigint, order_month: string, total_cnt: bigint, month_num: string
Sort [month_num#345 ASC NULLS FIRST], true
+- Aggregate [customer_id#2L, date_format(cast(order_date#1 as timestamp), MMMM, Some(America/Toronto))], [customer_id#2L, date_format(cast(order_date#1 as timestamp), MMMM, Some(America/Toronto)) AS order_month#342, count(1) AS total_cnt#343L, first(date_format(cast(order_date#1 as timestamp), MM, Some(America/Toronto)), false) AS month_num#345]
   +- SubqueryAlias order
      +- Relation[order_id#0L,order_date#1,customer_id#2L,order_status#3] csv

== Optimized Logical Plan ==
Sort [month_num#345 ASC NULLS FIRST], true
+- Aggrega